# Python Data Model

## Python Data Model

Reference: https://docs.python.org/3/reference/datamodel.html

### StaticArray Example

To demonstrate operator overloading, we'll implement a sequence type seen in other languages known as a *static array*:

- A static array is a sequence type (review: what makes a sequence type) where there is a fixed capacity to number of items the collection can hold.

- Resizing of the array is not allowed after initialization. 

- We will define a class ``StaticArray`` that will allow use to use built-in python operators.

We'll be able to use it like this:

```python
>>> from static_array import StaticArray
>>> sa = StaticArray([1, 2, 3])
>>> print(sa * 2)
# should produce the following output:
# [1, 2, 3, 1, 2, 3]
>>> print(sa[1])
2
```

In [ ]:
from collections.abc import Iterable


class StaticArray:
    def __init__(self, init_val, capacity=5):
        if isinstance(init_val, Iterable):
            self.items = list(init_val)
            self.capacity = len(self.items)
        else:
            self.items = [init_val] * capacity
            self.capacity = capacity


In [ ]:
sa = StaticArray([1, 2, 3])
print(sa)

In [ ]:
# we can fix that by defining a __repr__ method


class StaticArray:
    def __init__(self, init_val, capacity=5):
        if isinstance(init_val, Iterable):
            self.items = list(init_val)
            self.capacity = len(self.items)
        else:
            self.items = [init_val] * capacity
            self.capacity = capacity

    def __repr__(self):
        return f"StaticArray({self.items})"


sa = StaticArray([1, 2, 3])
print(sa)


### Emulating Collections & Sequences

**Collections**

- Have a length: `len(obj)`
- Can be iterated over: `for item in obj`
- Can query for membership: `item in obj`

**Sequences**

- Everything a collection can do
- Can be indexed: `obj[0]`

| You Write... | Python Calls... |
|--------------|-----------------|
| `len(obj)`   | `obj.__len__()` |
| `for item in obj` | `obj.__iter__()` |
| `item in obj` | `obj.__contains__(item)` |
| `obj[i]` | `obj.__getitem__(i)` |
| `obj[i] = x` | `obj.__setitem__(i, x)` |
| `del obj[i]` | `obj.__delitem__(i)` |


In [ ]:
class StaticArray:
    def __init__(self, init_val, capacity=5):
        if isinstance(init_val, Iterable):
            self.items = list(init_val)
            self.capacity = len(self.items)
        else:
            self.items = [init_val] * capacity
            self.capacity = capacity

    def __repr__(self):
        return f"StaticArray({self.items})"

    def __str__(self):
        return f"StaticArray({self.items})"

    def __len__(self):
        return self.capacity

    def __contains__(self, item):
        return item in self.items

    def __getitem__(self, index):
        if index >= self.capacity or index < -self.capacity:
            raise IndexError("Index out of range")
        return self.items[index]

    def __setitem__(self, index, val):
        if index >= self.capacity or index < -self.capacity:
            raise IndexError("Index out of range")
        self.items[index] = val

    def __delitem__(self, index):
        raise NotImplementedError("StaticArray does not support deletion")


In [ ]:
sa = StaticArray([1, "hi", 3.14, True])
len(sa)

In [ ]:
42 in sa
"hi" in sa

In [ ]:
sa[0]

In [ ]:
sa[42] = "hello"

### Numeric Operators

| You Write... | Python Calls... |
|--------------|-----------------|
| `x + y` | `x.__add__(y)` |
| `x - y` | `x.__sub__(y)` |
| `x * y` | `x.__mul__(y)` |
| `x / y` | `x.__truediv__(y)` |
| `x // y` | `x.__floordiv__(y)` |
| `x % y` | `x.__mod__(y)` |
| `x ** y` | `x.__pow__(y)` |
| `x @ y` | `x.__matmul__(y)` |

### Reverse / Reflected / Right Operators

| You Write... | Python Calls... |
|--------------|-----------------|
| `x + y` | `y.__radd__(x)` |
| `x - y` | `y.__rsub__(x)` |
| `x * y` | `y.__rmul__(x)` |
| `x / y` | `y.__rtruediv__(x)` |
| `x // y` | `y.__rfloordiv__(x)` |
| `x % y` | `y.__rmod__(x)` |
| `x ** y` | `y.__rpow__(x)` |
| `x @ y` | `y.__rmatmul__(x)` |

![](images/reverse_operators.png)

### Comparison Operators

| You Write... | Python Calls... |
|--------------|-----------------|  
| `x < y` | `x.__lt__(y)` |
| `x <= y` | `x.__le__(y)` |
| `x > y` | `x.__gt__(y)` |
| `x >= y` | `x.__ge__(y)` |
| `x == y` | `x.__eq__(y)` |
| `x != y` | `x.__ne__(y)` |


### Iteration

Iterable vs. Iterator (Review)

Objects like lists, tuples, and strings are *iterable*.

To keep track of the position within a given iteration (for loop, calls to `next`), Python uses an *iterator*.

In [ ]:
ll = [1, 2, 3, 4]
iterator = iter(ll)
print("iterator 1 next()", next(iterator))
print("iterator 1 next()", next(iterator))
iterator2 = iter(ll)
print("iterator 2 next()", next(iterator2))
print("iterator 1 next()", next(iterator))


To be iterable, a class needs an `__iter__` method that returns an iterator.

An iterator is an object with a `__next__` method that returns the next item in the iteration. It should raise `StopIteration` when there are no more items.

Common Pattern: If a class only needs to be iterable once, it can return itself as the iterator, thus fulfilling both roles.

In [ ]:
class SimpleRange:
    def __init__(self, n):
        self.current = 0
        self.n = n

    def __iter__(self):
        return self

    def __next__(self):
        if self.current >= self.n:
            raise StopIteration
        else:
            self.current += 1
            return self.current - 1

    def __repr__(self):
        return f"SimpleRange({self.n}, current={self.current})"


In [ ]:
sr = SimpleRange(5)
for i in sr:
    print(i)
next(sr)

In [ ]:
sr = SimpleRange(5)
iterator = iter(sr)
print("sr is iterator", sr is iterator)
print(sr)
next(sr)
print(sr)


#### Iteration Advice 

1. Do not implement the ``__next__()`` in a class that should only be an iterable. 
2. In order to support multiple traversals, the iterator must be a seperate object. 
3. A common design pattern is to delegate iteration to a seperate class that is iterable.

For example, defining an ``StaticArrayIterator`` class that is in charge iterating through the objects within an ``StaticArray`` object. 

In [ ]:
# Adding __iter__ to StaticArray
class StaticArrayIterator:
    def __init__(self, values):
        self.values = values
        self.position = 0

    def __next__(self):
        if self.position >= len(self.values):
            raise StopIteration
        item = self.values[self.position]
        self.position += 1
        return item

    def __repr__(self):
        return f"iterating over {self.values}, at position {self.position}"


# Adding __iter__


class StaticArray:
    def __init__(self, capacity, initial=None):
        self._items = [initial] * capacity
        self._capacity = capacity
        self._iter_position = 0

    @classmethod
    def from_iterable(self, iterable):
        new_array = StaticArray(len(iterable))
        for idx, item in enumerate(iterable):
            new_array._items[idx] = item
        return new_array

    def __repr__(self):
        # __repr__ is the unambiguous string representation
        # of an object
        return f"StaticArray({self._capacity}, {self._items})"

    def __str__(self):
        return repr(self._items)

    # Sequence Operations ####

    def __len__(self):
        return self._capacity

    def __contains__(self, x):
        return x in self._items

    def __getitem__(self, i):
        if i >= self._capacity or i < -self._capacity:
            raise IndexError  # an invalid index
        return self._items[i]

    def __setitem__(self, i, x):
        if i >= self._capacity or i < -self._capacity:
            raise IndexError  # an invalid index
        self._items[i] = x

    def __delitem__(self, i):
        raise NotImplementedError("Cannot delete from a static array")

    # Iterable Operations ####
    def __iter__(self):
        return StaticArrayIterator(self._items[:])


## Exceptions

In [ ]:
def divide(x, y):
    return x / y


In [ ]:
divide(1, 0)

In [ ]:
d = {}
print(d["bad_key"])

In [ ]:
l = [1, 2, 3]
print(l[42])

### Commonly Used Exceptions

- `AttributeError`
- `ImportError`
- `IndexError`
- `KeyError`
- `NameError`
- `NotImplementedError`
- `OSError`
- `StopIteration`
- `ValueError`
- `ZeroDivisionError`

https://docs.python.org/3/library/exceptions.html#BaseException

Generally not caught: 

- `SystemExit`
- `SyntaxError`

### Recovering From Exceptions

We can handle these exceptional cases via `try-except` blocks.

```python
try:
    statement1
    statement2
    ...
except ExceptionType as var:
    statement1
    statement2
    ...
```

In [ ]:
d = {}
try:
    print("before")
    print(d["bad_key"])
    print("after")
except KeyError as e:
    print("no such key", e)

In [ ]:
lst = []
try:
    print(lst[100])
except IndexError as e:  # capturing the exception
    print("error:", e)

In [ ]:
# catch multiple error types
d = []
try:
    print(d[0])
except IndexError as e:
    print("index error:", type(e), e)
except KeyError as e:
    print("key error:", type(e), e)

In [ ]:
# catching an exception's base class will catch any child exceptions

d = {}
try:
    print(d[0])
# Exception is a base class for all commonly caught exceptions
except Exception as e:
    print("error:", type(e))


### Defining & raising our own exceptions

Inherit from `Exception` or one of its subclasses.

Can provide custom constructor/attributes/etc. but often don't need to.

There is a default constructor that takes an (optional) message.

In [ ]:
class OddNumberException(ValueError):
    pass


def evens_only(n):
    if n % 2:
        raise OddNumberException()


### `else` and `finally`

```python
try:
    statement1
    statement2
    ...
except ExceptionType as var:
    # runs if exception was caught
except ExceptionType2 as var:
    # runs if exception was caught
else:
    # runs if no exception was raised
finally:
    # runs no matter what
```

In [ ]:
try:
    x = int(input("Enter a number:"))
    evens_only(x)
    print("only gets here if no exception is raised")
except OddNumberException:
    print("odd")
    if x == 7:
        raise
else:
    print("even")

print("our work here is done.")


Finally is often used to clean up after ourselves. Free resources. (e.g., close database connections, files)

## Context Handlers / `with`

We also saw this idea of needing to clean up after ourselves when we used `with` to open files.

```python

with open(filename) as f:
    # do things with f

# f is guaranteed to be closed even if 
# exceptions are raised within with block
```

### Yet another set of dunder methods...

In [ ]:
class DatabaseConnection:
    def __init__(self, username, password):
        # connect to database
        self.username = username
        self.password = password
        self.connected = True

    def __enter__(self):
        print("__enter__")
        # must return self!
        return self

    def __exit__(self, exc_type, exc_val, exc_traceback):
        print("__exit__")
        if exc_type:
            print("rolling back changes")
        self.connected = False

    def query(self, sql):
        print("ran query", sql)

    def __repr__(self):
        return f"Connection connected={self.connected}"


In [ ]:
db = DatabaseConnection("hello", "world")
db.query("SELECT * FROM users;")
# do something dangerous
1 / 0

In [ ]:
# our connection is possibly left in a broken state
print(db)

In [ ]:
with DatabaseConnection("hello", "world") as db:
    # __enter__
    db.query("SELECT * from users;")
    1 / 0
    # __exit__

In [ ]:
# changes were rolled back, and our connection is safe
db.connected


## Callable Objects Examples

Functions have a few attributes like `__name__` and `__doc__` that we can use to introspect on them.

In [ ]:
def add(x, y):
    """Adds two numbers"""
    return x + y


print(add.__name__)
print(add.__doc__)


They also have a `__call__` method that allows us to make our own objects callable. For example:

In [ ]:
class Memoized:
    def __init__(self, func):
        self.cache = {}
        self.wrapped_func = func

    def __call__(self, *args):
        if args not in self.cache:
            self.cache[args] = self.wrapped_func(*args)
        return self.cache[args]


In [ ]:
@Memoized
def expensive_func(a, b, c):
    print("running expensive_func")
    return a + b + c


print(expensive_func(1, 2, 3))
print(expensive_func(1, 2, 3))


In [ ]:
class PartialFunc:
    # simplified functools.partial

    def __init__(self, func, *args, **kwargs):
        self.func = func
        self.args = args
        self.kwargs = kwargs

    def __call__(self, *args, **kwargs):
        temp_kwargs = self.kwargs.copy()
        temp_kwargs.update(kwargs)
        return self.func(*(self.args + args), **temp_kwargs)

    @property
    def __name__(self):
        return f"{self.func.__name__}(args={self.args} kwargs={self.kwargs})"

    @property
    def __doc__(self):
        return self.func.__doc__


In [ ]:
def add(x, y):
    """Adds two numbers"""
    return x + y


add_5 = PartialFunc(add, 5)
print(add_5(10))

print(add_5.__name__)
print(add_5.__doc__)